# Détection de Fraude sur les Compteurs Électriques et Gaz

## Contexte
Une entreprise publique de distribution d'électricité et de gaz a subi d'énormes pertes en raison de manipulations frauduleuses des compteurs par des consommateurs. L'objectif est de détecter et d'identifier les clients impliqués dans des activités frauduleuses en utilisant l'historique de facturation.

## Méthodologie

### 1. Compréhension des Données
- **Données Client**: Informations démographiques et géographiques des clients
- **Données Factures**: Historique de consommation et informations sur les compteurs

### 2. Approche de Modélisation
1. **Exploration des données** (EDA)
2. **Ingénierie des features** - Agrégation des factures par client
3. **Prétraitement** - Encodage et normalisation
4. **Modélisation** - Classification binaire (Fraude vs Non-fraude)
5. **Évaluation** - Métriques de performance
6. **Prédiction** - Sur l'ensemble de test

In [ ]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Configuration de l'affichage
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

## Chargement des Données

In [ ]:
# Chargement des données
print("Chargement des données...")
client_train = pd.read_csv('data/client_train.csv')
invoice_train = pd.read_csv('data/invoice_train.csv')
client_test = pd.read_csv('data/client_test.csv')
invoice_test = pd.read_csv('data/invoice_test.csv')

print(f"Client Train: {client_train.shape}")
print(f"Invoice Train: {invoice_train.shape}")
print(f"Client Test: {client_test.shape}")
print(f"Invoice Test: {invoice_test.shape}")

## Analyse Exploratoire des Données (EDA)

In [ ]:
# Aperçu des données client
print("=== DONNÉES CLIENT ===")
print(client_train.head())
print("\nInformations:")
print(client_train.info())
print("\nStatistiques descriptives:")
print(client_train.describe())
print("\nValeurs manquantes:")
print(client_train.isnull().sum())

In [ ]:
# Distribution de la variable cible
print("\n=== DISTRIBUTION DE LA CIBLE ===")
print(client_train['target'].value_counts())
print(f"\nProportion de fraude: {client_train['target'].mean():.2%}")

# Visualisation
fig, ax = plt.subplots(1, 2, figsize=(12, 4))
client_train['target'].value_counts().plot(kind='bar', ax=ax[0], title='Distribution de la Cible')
ax[0].set_xlabel('Fraude (0=Non, 1=Oui)')
ax[0].set_ylabel('Nombre de Clients')
client_train['target'].value_counts(normalize=True).plot(kind='pie', ax=ax[1], 
                                                           autopct='%1.1f%%', 
                                                           title='Proportion de Fraude')
plt.tight_layout()
plt.show()

In [ ]:
# Aperçu des données factures
print("\n=== DONNÉES FACTURES ===")
print(invoice_train.head())
print("\nInformations:")
print(invoice_train.info())
print("\nStatistiques:")
print(invoice_train.describe())

## Ingénierie des Features

### Stratégie:
Les indicateurs de fraude potentiels incluent:
1. **Anomalies de consommation**: Variations inhabituelles, consommation très faible
2. **Problèmes de compteur**: Statut du compteur, remarques de lecture
3. **Comportement temporel**: Fréquence des factures, patterns de dates
4. **Coefficients**: Utilisation de coefficients de correction

In [ ]:
def create_invoice_features(invoice_df):
    """
    Créer des features agrégées à partir des données de facturation
    """
    # Copie pour éviter les modifications
    df = invoice_df.copy()
    
    # Conversion de la date
    df['invoice_date'] = pd.to_datetime(df['invoice_date'])
    
    # Calcul de la consommation totale
    df['total_consumption'] = (df['consommation_level_1'] + 
                               df['consommation_level_2'] + 
                               df['consommation_level_3'] + 
                               df['consommation_level_4'])
    
    # Différence d'index
    df['index_diff'] = df['new_index'] - df['old_index']
    
    # Consommation par mois
    df['consumption_per_month'] = df['total_consumption'] / (df['months_number'] + 0.01)
    df['index_diff_per_month'] = df['index_diff'] / (df['months_number'] + 0.01)
    
    # Features agrégées par client
    agg_features = df.groupby('client_id').agg({
        # Statistiques de consommation
        'total_consumption': ['mean', 'std', 'min', 'max', 'sum'],
        'consumption_per_month': ['mean', 'std', 'min', 'max'],
        'index_diff': ['mean', 'std', 'min', 'max'],
        'index_diff_per_month': ['mean', 'std'],
        
        # Statistiques par niveau de consommation
        'consommation_level_1': ['mean', 'sum'],
        'consommation_level_2': ['mean', 'sum'],
        'consommation_level_3': ['mean', 'sum'],
        'consommation_level_4': ['mean', 'sum'],
        
        # Informations sur le compteur
        'counter_statue': ['mean', 'std', 'nunique'],
        'counter_coefficient': ['mean', 'std', 'max'],
        'reading_remarque': ['mean', 'std', 'max'],
        'counter_code': ['nunique'],
        'counter_number': ['nunique'],
        
        # Informations temporelles
        'months_number': ['mean', 'sum', 'max'],
        'invoice_date': ['count', 'min', 'max']
    })
    
    # Aplatir les noms de colonnes
    agg_features.columns = ['_'.join(col).strip() for col in agg_features.columns.values]
    agg_features.reset_index(inplace=True)
    
    # Features additionnelles
    agg_features['invoice_count'] = agg_features['invoice_date_count']
    
    # Durée de la relation client (en jours)
    agg_features['client_duration_days'] = (
        agg_features['invoice_date_max'] - agg_features['invoice_date_min']
    ).dt.days
    
    # Fréquence moyenne des factures (jours entre factures)
    agg_features['avg_invoice_frequency'] = (
        agg_features['client_duration_days'] / (agg_features['invoice_count'] + 0.01)
    )
    
    # Ratio de consommation nulle (indicateur de fraude potentiel)
    zero_consumption = df.groupby('client_id')['total_consumption'].apply(
        lambda x: (x == 0).sum() / len(x)
    ).reset_index()
    zero_consumption.columns = ['client_id', 'zero_consumption_ratio']
    agg_features = agg_features.merge(zero_consumption, on='client_id', how='left')
    
    # Ratio de consommation très faible
    low_consumption = df.groupby('client_id')['total_consumption'].apply(
        lambda x: (x < x.quantile(0.1)).sum() / len(x)
    ).reset_index()
    low_consumption.columns = ['client_id', 'low_consumption_ratio']
    agg_features = agg_features.merge(low_consumption, on='client_id', how='left')
    
    # Volatilité de la consommation (indicateur de fraude)
    agg_features['consumption_volatility'] = (
        agg_features['total_consumption_std'] / (agg_features['total_consumption_mean'] + 0.01)
    )
    
    # Type de compteur majoritaire
    counter_type_mode = df.groupby('client_id')['counter_type'].agg(lambda x: x.mode()[0] if len(x.mode()) > 0 else 'UNKNOWN')
    counter_type_mode = counter_type_mode.reset_index()
    counter_type_mode.columns = ['client_id', 'counter_type_mode']
    agg_features = agg_features.merge(counter_type_mode, on='client_id', how='left')
    
    # Supprimer les colonnes de date
    agg_features = agg_features.drop(['invoice_date_min', 'invoice_date_max'], axis=1)
    
    return agg_features

print("Création des features d'entraînement...")
train_invoice_features = create_invoice_features(invoice_train)
print(f"Features créées: {train_invoice_features.shape}")
print("\nPremières lignes:")
print(train_invoice_features.head())

In [ ]:
print("Création des features de test...")
test_invoice_features = create_invoice_features(invoice_test)
print(f"Features créées: {test_invoice_features.shape}")

In [ ]:
# Fusion avec les données client
def prepare_client_features(client_df):
    """
    Préparer les features client
    """
    df = client_df.copy()
    
    # Conversion de la date de création
    df['creation_date'] = pd.to_datetime(df['creation_date'], format='%d/%m/%Y')
    
    # Features temporelles
    df['creation_year'] = df['creation_date'].dt.year
    df['creation_month'] = df['creation_date'].dt.month
    df['client_age_years'] = (datetime.now() - df['creation_date']).dt.days / 365.25
    
    # Supprimer la colonne date originale
    df = df.drop('creation_date', axis=1)
    
    return df

print("Préparation des données client...")
client_train_prep = prepare_client_features(client_train)
client_test_prep = prepare_client_features(client_test)

print("Client train préparé:")
print(client_train_prep.head())

In [ ]:
# Fusion des features
print("Fusion des features...")
X_train = client_train_prep.merge(train_invoice_features, on='client_id', how='left')
X_test = client_test_prep.merge(test_invoice_features, on='client_id', how='left')

print(f"Dataset d'entraînement: {X_train.shape}")
print(f"Dataset de test: {X_test.shape}")

# Sauvegarder les IDs de test
test_ids = X_test['client_id'].copy()

# Extraire la cible
y_train = X_train['target'].copy()
X_train = X_train.drop(['target', 'client_id'], axis=1)
X_test = X_test.drop(['client_id'], axis=1)

print(f"\nFeatures finales: {X_train.shape[1]}")
print(f"\nValeurs manquantes dans X_train:")
print(X_train.isnull().sum().sum())
print(f"\nValeurs manquantes dans X_test:")
print(X_test.isnull().sum().sum())

## Prétraitement des Données

In [ ]:
# Gestion des valeurs manquantes
print("Imputation des valeurs manquantes...")
X_train = X_train.fillna(X_train.median())
X_test = X_test.fillna(X_train.median())  # Utiliser les médianes du train

# Encodage des variables catégorielles
categorical_cols = X_train.select_dtypes(include=['object']).columns
print(f"\nColonnes catégorielles: {list(categorical_cols)}")

le_dict = {}
for col in categorical_cols:
    le = LabelEncoder()
    # Combiner train et test pour assurer la cohérence
    combined = pd.concat([X_train[col], X_test[col]], axis=0)
    le.fit(combined)
    X_train[col] = le.transform(X_train[col])
    X_test[col] = le.transform(X_test[col])
    le_dict[col] = le

print("Encodage terminé.")

# Vérification finale
print(f"\nX_train final: {X_train.shape}")
print(f"X_test final: {X_test.shape}")
print(f"y_train: {y_train.shape}")

In [ ]:
# Normalisation des features
print("Normalisation des features...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convertir en DataFrame pour garder les noms de colonnes
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

print("Normalisation terminée.")

## Modélisation

### Approche:
Nous utilisons un **Gradient Boosting Classifier** qui est particulièrement efficace pour:
- Les problèmes de classification binaire
- Les données déséquilibrées
- La capture de relations non-linéaires complexes
- L'importance des features

In [ ]:
# Split pour validation
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train_scaled, y_train, test_size=0.2, random_state=42, stratify=y_train
)

print(f"Training set: {X_tr.shape}")
print(f"Validation set: {X_val.shape}")
print(f"\nDistribution de la cible (training):")
print(y_tr.value_counts(normalize=True))

In [ ]:
# Entraînement du modèle Gradient Boosting
print("Entraînement du Gradient Boosting Classifier...")

gb_model = GradientBoostingClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=5,
    min_samples_split=100,
    min_samples_leaf=50,
    subsample=0.8,
    random_state=42,
    verbose=1
)

gb_model.fit(X_tr, y_tr)
print("\nEntraînement terminé!")

## Évaluation du Modèle

In [ ]:
# Prédictions sur validation
y_val_pred = gb_model.predict(X_val)
y_val_proba = gb_model.predict_proba(X_val)[:, 1]

# Métriques
print("=== PERFORMANCE SUR VALIDATION ===")
print("\nRapport de classification:")
print(classification_report(y_val, y_val_pred))

print(f"\nROC-AUC Score: {roc_auc_score(y_val, y_val_proba):.4f}")

# Matrice de confusion
cm = confusion_matrix(y_val, y_val_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Matrice de Confusion')
plt.ylabel('Vraie Classe')
plt.xlabel('Classe Prédite')
plt.show()

In [ ]:
# Importance des features
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': gb_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n=== TOP 20 FEATURES LES PLUS IMPORTANTES ===")
print(feature_importance.head(20))

# Visualisation
plt.figure(figsize=(10, 8))
plt.barh(range(20), feature_importance.head(20)['importance'])
plt.yticks(range(20), feature_importance.head(20)['feature'])
plt.xlabel('Importance')
plt.title('Top 20 Features les Plus Importantes')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## Prédiction sur le Test Set

Maintenant que le modèle est entraîné et validé, nous générons les prédictions finales sur l'ensemble de test.

In [ ]:
# Réentraîner sur toutes les données d'entraînement
print("Réentraînement sur toutes les données d'entraînement...")
final_model = GradientBoostingClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=5,
    min_samples_split=100,
    min_samples_leaf=50,
    subsample=0.8,
    random_state=42,
    verbose=1
)

final_model.fit(X_train_scaled, y_train)
print("Entraînement final terminé!")

In [ ]:
# Prédictions sur le test set
print("Génération des prédictions sur le test set...")
test_predictions_proba = final_model.predict_proba(X_test_scaled)[:, 1]

print(f"Prédictions générées: {len(test_predictions_proba)}")
print(f"\nDistribution des probabilités de fraude:")
print(pd.Series(test_predictions_proba).describe())

In [ ]:
# Visualisation de la distribution des prédictions
plt.figure(figsize=(10, 5))
plt.hist(test_predictions_proba, bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Probabilité de Fraude')
plt.ylabel('Nombre de Clients')
plt.title('Distribution des Probabilités de Fraude sur le Test Set')
plt.axvline(x=0.5, color='r', linestyle='--', label='Seuil de décision (0.5)')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

print(f"\nClients prédits comme frauduleux (prob > 0.5): {(test_predictions_proba > 0.5).sum()}")
print(f"Proportion: {(test_predictions_proba > 0.5).mean():.2%}")

## Sauvegarde des Résultats

In [ ]:
# Création du fichier de soumission
submission = pd.DataFrame({
    'client_id': test_ids,
    'target': test_predictions_proba
})

# Sauvegarder
submission.to_csv('SampleSubmission.csv', index=False)
print("Fichier de soumission sauvegardé: SampleSubmission.csv")
print(f"\nPremières lignes:")
print(submission.head(10))
print(f"\nDernières lignes:")
print(submission.tail(10))

## Conclusions et Recommandations

### Points Clés de la Solution:

1. **Ingénierie des Features**:
   - Agrégation des factures par client pour créer un profil de consommation
   - Calcul de métriques de volatilité et d'anomalie
   - Extraction de patterns temporels

2. **Indicateurs de Fraude**:
   - Consommation anormalement faible ou nulle
   - Forte volatilité de consommation
   - Statut de compteur problématique
   - Remarques de lecture négatives
   - Coefficients de correction élevés

3. **Modèle**:
   - Gradient Boosting pour capturer les relations complexes
   - Prédiction de probabilités pour permettre un ajustement du seuil
   - Performance évaluée avec ROC-AUC pour gérer le déséquilibre des classes

### Recommandations pour l'Entreprise:

1. **Priorisation**: Commencer par les clients avec probabilité > 0.7
2. **Inspection**: Vérifier physiquement les compteurs suspects
3. **Monitoring**: Suivre les patterns de consommation en temps réel
4. **Prévention**: Améliorer la technologie des compteurs (compteurs intelligents)
5. **Éducation**: Sensibiliser sur les conséquences légales de la fraude

### Améliorations Futures:

1. Collecte de plus de données historiques
2. Intégration de données géospatiales
3. Modèles d'apprentissage profond (Neural Networks)
4. Système de détection en temps réel
5. Feedback loop avec les résultats des inspections